In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
import torchvision

/home/briancy2/.conda/envs/iwildcam/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
META_DIR = "data/metadata/metadata/"
TRAIN_DIR = "data/train/train/"
TEST_DIR = "data/test/test/"

test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))
train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))

#Test images
df_test = pd.DataFrame({'id': [item['id'] for item in test_data['images']],
                                'seq_id': [item['seq_id'] for item in test_data['images']],
                                'file_name': [item['file_name'] for item in test_data['images']],
                                'location': [item['location'] for item in test_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in test_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in test_data['images']]})
#Train images
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['images']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})
# Detection for train test
detections = json.load(open(META_DIR+"iwildcam2022_mdv4_detections.json"))['images']
det_dict = dict()
for detection in tqdm(detections):
    det_dict[detection['file']] = detection['detections']
df_detection = pd.DataFrame({'file': [item['file'] for item in detections],
                                'detections': [item['detections'] for item in detections]})
# Test sequence ids
test_sequence_ids = pd.unique(df_test['seq_id'])
# Train sequence id and count
train_seq_count = pd.read_csv(META_DIR+"train_sequence_counts.csv")

100%|██████████| 261428/261428 [00:00<00:00, 1545939.36it/s]


In [3]:
train_seq_count.

SyntaxError: invalid syntax (1790939137.py, line 1)

In [5]:
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker

metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine", 0.2, 100)
file = open('submission_deep8.txt', 'w')
file.write("Id,Predicted")
for sequence_id in tqdm(test_sequence_ids):
    tracker = Tracker(metric)
    results = []
    img_rows = df_test.loc[df_test.seq_id == sequence_id]
    for index, img_row in img_rows.iterrows():
        detections = df_detection.loc[df_detection.file == ('test/'+img_row['file_name'])]['detections'].to_list()[0]
        img1 = im[:y, :]
        img2 = im[y+h:imh, :]
        
        #print(set([d["category"] for d in detections]))
        detections = [Detection(tuple(det["bbox"]), det["conf"], []) for det in detections if det['conf']]
        
#         boxes = np.array([d for d in filtered_det])
#         scores = np.array([d.confidence for d in detections])
        tracker.predict()
        tracker.update(detections)
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            results.append(track.track_id)
    count = len(set(results))   
    file.write('\n' + str(sequence_id) + ',' + str(count))

100%|██████████| 11028/11028 [19:35<00:00,  9.38it/s]


In [8]:
!git commit -am "baseline deep sort is 0.428 - absolutely horrible"

[main 6086694] baseline deep sort is 0.428 - absolutely horrible
 3 files changed, 325 insertions(+), 244 deletions(-)


In [9]:
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
print(len(train_seq_count))
file = open('submission_deep_test.txt', 'w')
file.write("Id,Predicted")
counts = []
metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine", 0.2, 100)
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    tracker = Tracker(metric)
    results = []
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    for index, img_row in img_rows.iterrows():
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        detections = [Detection(tuple(det["bbox"]), det["conf"], []) for det in detections]
#         boxes = np.array([d for d in filtered_det])
#         scores = np.array([d.confidence for d in detections])
        tracker.predict()
        tracker.update(detections)
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            results.append(track.track_id)
    count = len(set(results))   
    counts.append(count)
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))
    

1780


1780it [05:50,  5.07it/s]
